In [11]:
import tensorflow as tf
import tensorflow.keras as keras
import json
import numpy as np
import music21 as m21


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/melody_generator_lstm/Data_pre_processing'

/content/drive/MyDrive/Colab Notebooks/melody_generator_lstm/Data_pre_processing


In [4]:
SEQUENCE_LENGTH = 64
MAPPING_PATH = "mapping.json"

In [14]:
from os import supports_effective_ids
class MelodyGenerator:
    def __init__(self, model_path="model.h5"):
        self.model_path = model_path
        self.model = keras.models.load_model(self.model_path)
        with open(MAPPING_PATH, "r") as f:
            self._mappings = json.load(f)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH

    def _sample_with_temperature(self, probabilities, temperature):
        predictions = np.log(probabilities) / temperature
        predictions = np.exp(predictions)
        predictions = predictions / np.sum(predictions)
        return np.random.choice(len(self._mappings), p=predictions)

    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):
          seed = seed[-max_sequence_length:]
          onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
          onehot_seed = onehot_seed[np.newaxis, ...]
          probabilities = self.model.predict(onehot_seed)[0]
          output_int = self._sample_with_temperature(probabilities, temperature)
          seed.append(output_int)
          output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]
          if output_symbol == "/":
            break
          melody.append(output_symbol)
        return melody
    def save_melody(self, melody, step_duration=0.25, format = "midi", file_name="mel.midi"):
        stream = m21.stream.Stream()
        start_symbol = None
        step_counter = 1

        for i, symbol in enumerate(melody):
            if symbol != "_" or i + 1 == len(melody):
              if start_symbol is not None:
                quarter_length_duration = step_duration * step_counter
                if start_symbol == "r":
                  m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
                else:
                  m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
                stream.append(m21_event)
                step_counter = 1
                start_symbol = symbol

            else:
                step_counter += 1
        stream.write(format, file_name)


In [15]:
mg = MelodyGenerator()
seed = "76 _ _ _ 77 _ _ _ 77 _ _ _ 79 _ _ _ 79 _ _ _ 81 _"
melody = mg.generate_melody(seed, 500, SEQUENCE_LENGTH, 0.7)
print(melody)
mg.save_melody(melody)

1/1 [==============================] - 0s 52ms/step


<ipython-input-14-f9220141afa5>:12: RuntimeWarning: divide by zero encountered in log
  predictions = np.log(probabilities) / temperature


1/1 [==============================] - 0s 38ms/step
['76', '_', '_', '_', '77', '_', '_', '_', '77', '_', '_', '_', '79', '_', '_', '_', '79', '_', '_', '_', '81', '_', '_', '_', '76', '_', '_', '_', 'r', '_', '76', '_', '81', '_', '_', '_', '_', '_', '81', '_', '81', '_', '_', '_', '_', '_', '_', '_', '76', '_', '_', '_', '74', '_', '_', '_', '76', '_', '_', '_', '74', '_', '_', '_', '72', '_', '_', '_', '71', '_', '_', '_', '69', '_', '_', '_', '71', '_', '_', '_', '72', '_', '_', '_', '74', '_', '_', '_', '76', '_', '_', '_', 'r', '_', '76', '_', '74', '_', '_', '_', '74', '_', '72', '_', '71', '_', '_', '_', '_', '_', '_', '_', '71', '_', '_', '_', '71', '_', '_', '_', '72', '_', '_', '_', '71', '_', '_', '_', '69', '_', '_', '_', '68', '_', '_', '_', '69', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']
